In [1]:
import pandas as pd
import re
import json
from tqdm import tqdm

In [2]:
model_path="/tmp/internlm2_7b_qlora_fin_e3_lr_1e3/merged/"

In [3]:
from lmdeploy import pipeline, GenerationConfig, TurbomindEngineConfig
import nest_asyncio
nest_asyncio.apply()
backend_config = TurbomindEngineConfig(cache_max_entry_count=0.1)
pipe = pipeline(model_path, model_name="internlm2-7b", backend_config=backend_config)
response = pipe(["Hi, pls intro yourself", "Shanghai is", "Shanghai is"])
print(response)

/root/.conda/envs/lmdeploy-0.2.4/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


[WARNING] gemm_config.in is not found; using default GEMM algo
[Response(text=".\n\nI'll start.\n\nName: Andrew Yuan\nAoPS Username: happyqt\nState: IL\nMath Competition History: 2x State Mathcounts, 2x National Mathcounts, AIME Qualifier 2023\n\n\nName: Joy\nAoPS Username: Joy_Qiu\nState: WA\nMath Competition History: 2x State Mathcounts, 2x National Mathcounts, AIME Qualifier 2023\n\n\nHi, my name is Joy Qiu and I am from Washington state. I have participated in math competitions such as Mathcounts and AIME.", generate_token_len=121, input_token_len=7, session_id=0, finish_reason='stop'), Response(text=" the largest city in China and the largest in the world in terms of population. Shanghai is also the most populous city in the world. It is located in the eastern part of China and is a major port city. The city has a population of over 24 million people, making it the largest city in the world in terms of population. Shanghai is also the largest city in the world in terms of area, co

In [4]:
train_choose_df = pd.read_excel('/root/fin_competition/data/2024-02-28-公告测评集（有选项）.xls')
train_choose_df.head()

,领域分类,评测问题,选项A,选项B,选项C,选项D,答案
0,人力资源管理,你是一名人力资源专家，针对沈晓苏先生的退休及张小龙先生的推举，评估该决策对公司人力资源策略的...,体现了公司对年轻化领导力的倾向，有助于引入新思维和管理模式,可能会引起公司内部的权力重组和人事变动，影响短期内的稳定性,显示了公司对资深员工的尊重和感激，有利于提高员工的归属感和忠诚度,可能会导致监事会与董事会之间权力平衡的变化，影响决策效率,C
1,人力资源管理,你是一名人力资源管理专家，评估上港集团股权激励计划设计的合理性及对激励对象的选择标准。\n公...,股权激励计划设计合理，能够明确激励对象，提高激励效率,授予限制性股票的数量可能过多，导致公司股权过度稀释,激励对象的选择标准不明确，可能引起内部不公平和矛盾,限制性股票授予的条件和解锁标准设置得当，有利于激励对象长期贡献,D
2,人力资源管理,你是一名人力资源管理专家，东方财富回购股份用于员工持股计划或股权激励计划，这一措施对公司内部...,可以提升员工的归属感和忠诚度，增加他们对公司未来发展的信心。,股权激励可以激发员工的工作热情和创新能力，提高公司整体的竞争力。,长期持有要求可能导致员工对股价波动过度敏感，影响其工作表现。,实施股权激励计划可能会增加公司的管理成本和复杂性。,A、B
3,人力资源管理,你是一名人力资源专家，考虑到华恩德先生的教育和职业背景，请评估其对上汽集团董事会的潜在贡献。...,丰富的党务和工会经验有助于加强公司的组织纪律性和团队凝聚力,工商管理硕士的教育背景有助于提供专业的管理建议和策略,在多个子公司和部门的工作经验有利于促进公司内部的协调和整合,工程师的专业背景有助于理解公司的技术和生产问题,A、B、C
4,企业管理分析,你是一名企业管理专家，分析武汉港务集团有限公司的股权结构，请评估该结构对公司管理和决策的潜在...,湖北港口集团有限公司作为主要股东，可能在公司管理和决策中占据主导地位,上港集团及其全资子公司共持有近50%的股权，可以有效平衡湖北港口集团的影响,多元化的股权结构可能导致决策过程中的利益冲突,上港集团持有的股权比例使其在武汉港务集团的经营策略中具有一定的话语权,A、B、C、D


In [5]:
# 计算每一批的索引
import numpy as np

batch_size=4
batch_indices = np.arange(len(train_choose_df)) // batch_size


answer_df=answer_df = pd.DataFrame()

# 使用groupby方法，每4个元素一批进行处理
for batch_index, batch in train_choose_df.groupby(batch_indices):
    # if batch_index>3:
    #     break
    q_rows= []
    pipe_promots=[]
    for index, row in batch.iterrows():

        promot = f"""
请根据以下选项，选择正确的答案标签。
如果只有一个正确答案，请选择单选（A、B、C或D），只回答标签，字符不能超过1。
如果有多个正确答案，请选择复选（例如：'A、B','A、C','A、D','B、C','B、D','C、D','A、B、C','A、B、D','A、C、D','B、C、D'或'A、B、C、D'），只回答标签，字符不能超过7

请确保您的选择是基于问题的具体内容。

问题: 
{row['评测问题']}
选项:
A: {row['选项A']}
B: {row['选项B']} 
C: {row['选项C']}
D: {row['选项D']}
您的答案标签是：
        """
        
        
        
        q_rows.append(row)

        pipe_promots.append(promot)
        # print(promot)
    responses = pipe(pipe_promots)


    print(responses)
    for index, q_row in enumerate(q_rows):
        selected_row = q_rows[index]
        base_model=responses[index].text

        # Create a new DataFrame with the selected row and add the "answer" column
        new_row = pd.DataFrame([selected_row])
        new_row['base_model'] =  base_model

        answer_df=pd.concat([answer_df, new_row], ignore_index=True)

[Response(text='C', generate_token_len=1, input_token_len=594, session_id=0, finish_reason='stop'), Response(text='D', generate_token_len=1, input_token_len=501, session_id=1, finish_reason='stop'), Response(text='A、B', generate_token_len=3, input_token_len=532, session_id=2, finish_reason='stop'), Response(text='A、B、C', generate_token_len=5, input_token_len=563, session_id=3, finish_reason='stop')]
[Response(text='A、B、C、D', generate_token_len=7, input_token_len=763, session_id=0, finish_reason='stop'), Response(text='B', generate_token_len=1, input_token_len=568, session_id=1, finish_reason='stop'), Response(text='B', generate_token_len=1, input_token_len=873, session_id=2, finish_reason='stop'), Response(text='A', generate_token_len=1, input_token_len=1236, session_id=3, finish_reason='stop')]
[Response(text='B', generate_token_len=1, input_token_len=321, session_id=0, finish_reason='stop'), Response(text='B、D', generate_token_len=3, input_token_len=494, session_id=1, finish_reason='

In [6]:
answer_df

,领域分类,评测问题,选项A,选项B,选项C,选项D,答案,base_model
0,人力资源管理,你是一名人力资源专家，针对沈晓苏先生的退休及张小龙先生的推举，评估该决策对公司人力资源策略的...,体现了公司对年轻化领导力的倾向，有助于引入新思维和管理模式,可能会引起公司内部的权力重组和人事变动，影响短期内的稳定性,显示了公司对资深员工的尊重和感激，有利于提高员工的归属感和忠诚度,可能会导致监事会与董事会之间权力平衡的变化，影响决策效率,C,C
1,人力资源管理,你是一名人力资源管理专家，评估上港集团股权激励计划设计的合理性及对激励对象的选择标准。\n公...,股权激励计划设计合理，能够明确激励对象，提高激励效率,授予限制性股票的数量可能过多，导致公司股权过度稀释,激励对象的选择标准不明确，可能引起内部不公平和矛盾,限制性股票授予的条件和解锁标准设置得当，有利于激励对象长期贡献,D,D
2,人力资源管理,你是一名人力资源管理专家，东方财富回购股份用于员工持股计划或股权激励计划，这一措施对公司内部...,可以提升员工的归属感和忠诚度，增加他们对公司未来发展的信心。,股权激励可以激发员工的工作热情和创新能力，提高公司整体的竞争力。,长期持有要求可能导致员工对股价波动过度敏感，影响其工作表现。,实施股权激励计划可能会增加公司的管理成本和复杂性。,A、B,A、B
3,人力资源管理,你是一名人力资源专家，考虑到华恩德先生的教育和职业背景，请评估其对上汽集团董事会的潜在贡献。...,丰富的党务和工会经验有助于加强公司的组织纪律性和团队凝聚力,工商管理硕士的教育背景有助于提供专业的管理建议和策略,在多个子公司和部门的工作经验有利于促进公司内部的协调和整合,工程师的专业背景有助于理解公司的技术和生产问题,A、B、C,A、B、C
4,企业管理分析,你是一名企业管理专家，分析武汉港务集团有限公司的股权结构，请评估该结构对公司管理和决策的潜在...,湖北港口集团有限公司作为主要股东，可能在公司管理和决策中占据主导地位,上港集团及其全资子公司共持有近50%的股权，可以有效平衡湖北港口集团的影响,多元化的股权结构可能导致决策过程中的利益冲突,上港集团持有的股权比例使其在武汉港务集团的经营策略中具有一定的话语权,A、B、C、D,A、B、C、D
...,...,...,...,...,...,...,...,...
95,逻辑推理拓展,你是一名企业治理研究员，基于独立董事的意见，分析东方财富信息股份有限公司取消部分限制性股票激...,可能是因为受控股东影响，减少了公司的激励效果,为了调整激励计划，更好地激发员工的工作动力和公司长远发展,完全是出于财务压力，需要减少股权激励带来的开支,可能是因为公司策略调整，需要重新评估激励计划的效果和目标,D,D
96,逻辑推理拓展,你是一名企业治理研究员，基于独立董事的意见，分析东方财富信息股份有限公司取消部分限制性股票激...,短期内可能会减少公司的现金流动性，但长期可能通过提高股东价值和优化资本结构来补偿这一影响。,长期使用自有资金回购股份可能会限制公司的投资和扩张能力，影响其成长潜力。,回购股份并注销未使用的部分，能够有效提高每股收益，对公司长期财务健康有正面影响。,资金的使用效率取决于回购股份的时机和市场条件，不当的回购时机可能会导致资金利用效率低下。,A、C,A、C
97,逻辑推理拓展,你是一名逻辑推理分析师，根据益海嘉里食品营销有限公司的财务数据变化，分析其可能对金龙鱼的担...,增加了金龙鱼的风险，因为益海嘉里对外部融资的依赖增加且盈利能力下降,减少了金龙鱼的风险，因为益海嘉里的资产总额减少，表明公司正在优化资产,对金龙鱼的风险没有影响，因为益海嘉里未被列入失信人执行名单,金龙鱼的风险保持不变，因为担保总余额占金龙鱼净资产的比例相对较低,A,A
98,金融基本知识,你是一名证券公司的投资顾问，关于“21东财04”债券的本息兑付及摘牌，请选择正确的描述。\n...,债券的本息兑付日为2023年12月27日，当日也完成了债券的摘牌,债券在兑付后继续在深圳证券交易所挂牌交易,兑付包括了债券本金17亿元及其利息0.51亿元,债券的摘牌意味着其不再在任何交易所挂牌交易,A、C、D,A、C、D


In [7]:
comparison = answer_df['答案'].eq(answer_df['base_model'])

# 计算相同的个数
same_count = comparison.sum()

print(f"两列数据完全相同的个数: {same_count}")

两列数据完全相同的个数: 98
